In [1]:
# %pip install llama-index-vector-stores-mongodb

In [3]:
# MODEL_REPO = "thenlper/gte-small"
TEXT_GENERATION_MODEL_REPO = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# TEXT_GENERATION_MODEL_REPO = "HuggingFaceH4/zephyr-7b-alpha"
# MODEL_REPO = "Writer/camel-5b-hf"
from my_settings import LLMSettings

model = LLMSettings(
    model_name=TEXT_GENERATION_MODEL_REPO,
    query_wrapper_prompt="""
    Below is an instruction that describes a task. 
    Write a response that appropriately completes the request.\n\n
    ### Instruction:\n{query_str}\n\n### Response:
""",
)

model._set_huggingface_embedding(local=True)
model._setup_huggingface_llm_inference_api()
model._configure_mongodb()

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\Dhruv Patel\.cache\huggingface\token
Login successful
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [4]:
import pymongo
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext

mongodb_client = pymongo.MongoClient(model.MONGO_DB_URI)
atlas_vector_search = MongoDBAtlasVectorSearch(
    mongodb_client,
    db_name = "llamaindex_db",
    collection_name = "test",
    index_name = "vector_index"
)
storage_context = StorageContext.from_defaults(vector_store=atlas_vector_search)

In [11]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex,load_index_from_storage


docs = SimpleDirectoryReader(input_files=["./data/nimrethanda.pdf"]).load_data()
print(docs)
vector_store_index = VectorStoreIndex.from_documents(
    documents=docs, storage_context=storage_context, show_progress=True
)
# vector_store_index=load_index_from_storage(storage_context=storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.


ValueError: No index in storage context, check if you specified the right persist_dir.

In [ ]:
# retriever = vector_store_index.as_retriever(similarity_top_k=3)
# nodes = retriever.retrieve("flower")

# for node in nodes:
#     print(node)

In [12]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# Instantiate Atlas Vector Search as a retriever
vector_store_retriever = VectorIndexRetriever(index=vector_store_index, similarity_top_k=5)

# Pass the retriever into the query engine
query_engine = RetrieverQueryEngine(retriever=vector_store_retriever)

# Prompt the LLM
# response = query_engine.query('write 5 flowers name grows more than 2500m height from provided in context only')
response = query_engine.query('tell me about diff names of GENTIAN')

print(response)
# print("\nSource documents: ")
# print(response.source_nodes)

AttributeError: 'MongoDBAtlasVectorSearch' object has no attribute 'vector_store'